In [5]:
import pandas as pd
import numpy as np
import tpot as tp

In [6]:
inputData = pd.read_csv("train.csv")
inputData.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
inputData.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [8]:
#inputData = inputData.dropna()
inputData = inputData[['Pclass','Sex','Age','Fare','Embarked','Survived']]
inputData = inputData.replace(r'^\s*$', np.nan, regex=True)
print(inputData.isna().sum())
inputData['Age'] = inputData['Age'].fillna(inputData['Age'].mean())
inputData['Pclass'] = inputData['Pclass'].fillna(inputData['Pclass'].median())
inputData['Embarked'] = inputData['Embarked'].fillna('S')

Pclass        0
Sex           0
Age         177
Fare          0
Embarked      2
Survived      0
dtype: int64


In [9]:
print(inputData.isna().sum())

Pclass      0
Sex         0
Age         0
Fare        0
Embarked    0
Survived    0
dtype: int64


In [10]:
inputData.head()

,Pclass,Sex,Age,Fare,Embarked,Survived
0,3,male,22.0,7.2500,S,0
1,1,female,38.0,71.2833,C,1
2,3,female,26.0,7.9250,S,1
3,1,female,35.0,53.1000,S,1
4,3,male,35.0,8.0500,S,0


In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [12]:
inputData[['Pclass','Embarked','Sex']].head()

,Pclass,Embarked,Sex
0,3,S,male
1,1,C,female
2,3,S,female
3,1,S,female
4,3,S,male


In [13]:
le = LabelEncoder()
transformedData = inputData[['Pclass','Embarked','Sex']].apply(le.fit_transform)
transformedData.head()

,Pclass,Embarked,Sex
0,2,2,1
1,0,0,0
2,2,2,0
3,0,2,0
4,2,2,1


In [14]:
oneHot = OneHotEncoder(sparse=False,handle_unknown='ignore')
transformedData = pd.DataFrame(oneHot.fit_transform(transformedData))
transformedData.head()

,0,1,2,3,4,5,6,7
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


In [15]:
transformedData[ 'Age' ] = inputData.Age
transformedData[ 'Fare' ] = inputData.Fare

In [16]:
transformedData.head()

,0,1,2,3,4,5,6,7,Age,Fare
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,22.0,7.2500
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,38.0,71.2833
2,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,26.0,7.9250
3,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,35.0,53.1000
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,35.0,8.0500


In [17]:
print(transformedData.shape)
print(inputData.shape)

(891, 10)
(891, 6)


In [18]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

In [67]:
X_train, X_test, y_train, y_test = train_test_split(transformedData, inputData['Survived'],
                                                    train_size=0.80, test_size=0.20)

In [68]:
pipeline_optimizer = TPOTClassifier(generations=3,population_size=70, cv=3,
                                    random_state=42, verbosity=3,  n_jobs=-1,max_eval_time_mins=1)
pipeline_optimizer.fit(X_train, y_train)

30 operators have been imported by TPOT.


_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required..
_pre_test decorator: _random_mutation_operator: num_test=0 feature_names mismatch: ['0', '1', '2', '3', '4', '5', '6', '7', 'Age', 'Fare'] ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9']
expected 6, 5, 4, 3, 2, Age, 0, 7, Fare, 1 in input data
training data did not have the following fields: f5, f1, f8, f6, f2, f3, f9, f4, f7, f0.
_pre_test decorator: _random_mutation_operator: num_test=0 unhashable type: 'slice'.
_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required..
_pre_test decorator: _random_mutation_operator: num_test=0 Expected n_neighbors <= n_samples,  but n_samples = 50, n_neighbors = 93.
_pre_test decorator: _random_mutation_operator: num_test=0 unhashable type: 'slice'.
_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported 

TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=3,
        disable_update_check=False, early_stop=None, generations=3,
        max_eval_time_mins=1, max_time_mins=None, memory=None,
        mutation_rate=0.9, n_jobs=-1, offspring_size=None,
        periodic_checkpoint_folder=None, population_size=70,
        random_state=42, scoring=None, subsample=1.0, template=None,
        use_dask=False, verbosity=3, warm_start=False)

In [65]:
print(pipeline_optimizer.score(X_test, y_test))

0.8268156424581006


In [66]:
pipeline_optimizer.export('tpot_exported_pipeline.py')

In [ ]:
import json
with open('data.txt', 'w') as outfile:
    json.dump(pipeline_optimizer.evaluated_individuals_, outfile)
